In [1]:
import logging
import sys

from naslib.defaults.trainer import Trainer

from naslib.search_spaces import (
    DartsSearchSpace,
    SimpleCellSearchSpace,
    NasBench101SearchSpace,
    HierarchicalSearchSpace,
)
from naslib.search_spaces.nasbench101 import graph

from naslib.utils import get_dataset_api, setup_logger, utils

import numpy as np

device: cpu
device: cpu
device: cpu
device: cpu
device: cpu
device: cpu


### Defining the Dataset

In [2]:
config = utils.get_config_from_args(config_type="nas")

In [3]:
dataset_api = get_dataset_api(config.search_space, config.dataset)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 5 seconds


### Defining the Global Variables

In [4]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"
OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]

NUM_VERTICES = 7
OP_SPOTS = NUM_VERTICES - 2
MAX_EDGES = 9

In [5]:
def sample_random_architecture(dataset_api, arch_limit=10):
        """
        This will sample a random architecture and update the edges in the
        naslib object accordingly.
        From the NASBench repository:
        one-hot adjacency matrix
        draw [0,1] for each slot in the adjacency matrix
        """
        architectures = []
        while len(architectures) < arch_limit:
            matrix = np.random.choice([0, 1], size=(NUM_VERTICES, NUM_VERTICES))
            matrix = np.triu(matrix, 1)
            ops = np.random.choice(OPS, size=NUM_VERTICES).tolist()
            ops[0] = INPUT
            ops[-1] = OUTPUT
            spec = dataset_api["api"].ModelSpec(matrix=matrix, ops=ops)
            if dataset_api["nb101_data"].is_valid(spec):
                architectures.append({"matrix": matrix, "ops": ops})
                #break
        
        return architectures
            
        #self.set_spec({"matrix": matrix, "ops": ops})

In [6]:
sampled_architectures = sample_random_architecture(dataset_api)

In [7]:
sampled_architectures

[{'matrix': array([[0, 0, 0, 1, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'maxpool3x3',
   'conv1x1-bn-relu',
   'maxpool3x3',
   'conv1x1-bn-relu',
   'maxpool3x3',
   'output']},
 {'matrix': array([[0, 1, 1, 1, 1, 1, 0],
         [0, 0, 1, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'maxpool3x3',
   'maxpool3x3',
   'conv3x3-bn-relu',
   'conv3x3-bn-relu',
   'maxpool3x3',
   'output']},
 {'matrix': array([[0, 0, 1, 0, 0, 0, 1],
         [0, 0, 0, 1, 0, 0, 1],
         [0, 0, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'maxpool3x3'

In [8]:
sampled_architectures[0]

{'matrix': array([[0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]]),
 'ops': ['input',
  'maxpool3x3',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'output']}

In [69]:
sampled_architectures[0].parse()

AttributeError: 'dict' object has no attribute 'parse'

In [63]:
from naslib.predictors.utils.models import nasbench1 as nas101_arch
from naslib.predictors.utils.models import nasbench1_spec
import torchvision.transforms as transforms
import torchvision.datasets as dset
from pathlib import Path

In [66]:
def get_project_root() -> Path:
    """
    Returns the root path of the project.
    """
    return Path('./').parent.parent

In [49]:
spec = nasbench1_spec._ToModelSpec(
    sampled_architectures[0]["matrix"], sampled_architectures[0]["ops"]
)

num_classes_dic = {"cifar10": 10, "cifar100": 100, "ImageNet16-120": 120}

network = nas101_arch.Network(
    spec,
    stem_out=128,
    num_stacks=3,
    num_mods=3,
    num_classes=num_classes_dic["cifar10"]
)

In [48]:
type(network)

naslib.predictors.utils.models.nasbench1.Network

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [54]:
on_device = network.to(device)

In [57]:
type(on_device)

naslib.predictors.utils.models.nasbench1.Network

In [59]:
class Cutout(object):
    def __init__(self, length, prob=1.0):
        self.length = length
        self.prob = prob

    def __call__(self, img):
        if np.random.binomial(1, self.prob):
            h, w = img.size(1), img.size(2)
            mask = np.ones((h, w), np.float32)
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1:y2, x1:x2] = 0.0
            mask = torch.from_numpy(mask)
            mask = mask.expand_as(img)
            img *= mask
        return img


In [60]:

CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ]
)

train_transform.transforms.append(Cutout(16, 1))

valid_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ]
)


In [70]:
data = "{}/data".format(get_project_root())
seed = 42

In [71]:
train_data = dset.CIFAR10(
    root=data, train=True, download=True, transform=train_transform
)
test_data = dset.CIFAR10(
    root=data, train=False, download=True, transform=valid_transform
)

Files already downloaded and verified
Files already downloaded and verified


In [74]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.8 * num_train))

train_queue = torch.utils.data.DataLoader(
    train_data,
    batch_size=100,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed+1),
)

valid_queue = torch.utils.data.DataLoader(
    train_data,
    batch_size=100,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:num_train]),
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed),
)

test_queue = torch.utils.data.DataLoader(
    test_data,
    batch_size=100,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed),
)

In [36]:
from naslib.optimizers import (
    DARTSOptimizer,
    GDASOptimizer,
    DrNASOptimizer,
    RandomSearch,
    RegularizedEvolution,
    LocalSearch,
    BasePredictor,
)

In [37]:
optimizer = RandomSearch(config)

In [38]:
search_space = NasBench101SearchSpace()

In [12]:
optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

In [13]:
model = optimizer.new_epoch(108)

In [14]:
model

Module(
  (arch): Graph makrograph-0.0963693, scope macro, 6 nodes
)

In [26]:
optimizer.sampled_archs

[Module(
   (arch): Graph makrograph-0.0963693, scope macro, 6 nodes
 )]

In [33]:
search_space.nodes

NodeView((1, 2, 3, 4, 5, 6))

In [35]:
for node_idx in lexicographical_topological_sort(search_space): 
    if "subgraph" in search_space.nodes[node_idx]:
                search_space.nodes[node_idx]["subgraph"].parse()
                search_space.add_module(
                    "{}-subgraph_at({})".format(search_space.name, node_idx),
                    search_space.nodes[node_idx]["subgraph"],
                )
    else:
        if isinstance(search_space.nodes[node_idx]["comb_op"], torch.nn.Module):
            search_space.add_module(
                "{}-comb_op_at({})".format(search_space.name, node_idx),
                search_space.nodes[node_idx]["comb_op"],
            )
            
    for neigbor_idx in search_space.neighbors(node_idx):
            edge_data = search_space.get_edge_data(node_idx, neigbor_idx)
            if isinstance(edge_data.op, Graph):
                edge_data.op.parse()
            elif edge_data.op.get_embedded_ops():
                for primitive in edge_data.op.get_embedded_ops():
                    if isinstance(primitive, Graph):
                        primitive.parse()
            search_space.add_module(
                "{}-edge({},{})".format(search_space.name, node_idx, neigbor_idx),
                edge_data.op,
            )

NameError: name 'Graph' is not defined

In [16]:
from networkx.algorithms.dag import lexicographical_topological_sort

In [17]:
import torch

In [28]:
for node_idx in lexicographical_topological_sort(optimizer.sampled_archs[0].arch):
            if "subgraph" in model.nodes[node_idx]:
                model.nodes[node_idx]["subgraph"].parse()
                model.add_module(
                    "{}-subgraph_at({})".format(model.name, node_idx),
                    model.nodes[node_idx]["subgraph"],
                )
            else:
                if isinstance(model.nodes[node_idx]["comb_op"], torch.nn.Module):
                    model.add_module(
                        "{}-comb_op_at({})".format(model.name, node_idx),
                        model.nodes[node_idx]["comb_op"],
                    )
            for neigbor_idx in model.neighbors(node_idx):
                edge_data = model.get_edge_data(node_idx, neigbor_idx)
                edge_data.op.parse()
                if edge_data.op.get_embedded_ops():
                    for primitive in edge_data.op.get_embedded_ops():
                        primitive.parse()
                model.add_module(
                    "{}-edge({},{})".format(model.name, node_idx, neigbor_idx),
                    edge_data.op,
                )
        #model.is_parsed = True

AttributeError: 'Module' object has no attribute 'nodes'